In [1]:
import smax

In [3]:
src = smax.SmaxRedisClient("localhost")

In [4]:
dic = {
     '1':'test 1',
     '2':{
         '2.1':'test 2.1',
         '2.2':{
             '2.2.1':'test 2.2.1',
             '2.2.2':'test 2.2.2'
             },
         '2.3':{'2.3.1':'test 2.3.1'}
         },
     '3':{'3.1':'test 3.1'}
     }

In [247]:
table = "test_share_struct"
expected_temp_value1 = 100
expected_temp_value2 = 0
expected_firmware_value1 = 2.0
expected_firmware_value2 = 2.1
expected_type_temp = int
expected_dim_temp = 1
expected_type_firmware = float
expected_dim_firmware = 1

struct = {"roach2-03": {"temp": expected_temp_value1, "firmware": expected_firmware_value1},
          "roach2-04": {"temp": expected_temp_value2, "firmware": expected_firmware_value2}}

In [13]:
dic

{'1': 'test 1',
 '2': {'2.1': 'test 2.1',
  '2.2': {'2.2.1': 'test 2.2.1', '2.2.2': 'test 2.2.2'},
  '2.3': {'2.3.1': 'test 2.3.1'}},
 '3': {'3.1': 'test 3.1'}}

In [258]:
def _recurse_nested_dict(dictionary):
    """
    Private function to recursively traverse a nested dictionary, finding
    the leaf nodes that have actual data values.  Each real data value
    is yielded back as it recurses.
    Args:
        dictionary (dict): Dict containing keys that exist in SMAX.

    Yields:
        (key, value) for every leaf node in the nested dictionary.
    """
    for key, value in dictionary.items():
        if isinstance(value, dict):
            # If value is dict then iterate over all its values
            for pair in _recurse_nested_dict(value):
                yield (f"{key}:{pair[0]}", *pair[1:])
            
        else:
            yield key, value

In [268]:
leaves = list(_recurse_nested_dict(dic))

In [269]:
leaves

[('1', 'test 1'),
 ('2:2.1', 'test 2.1'),
 ('2:2.2:2.2.1', 'test 2.2.1'),
 ('2:2.2:2.2.2', 'test 2.2.2'),
 ('2:2.3:2.3.1', 'test 2.3.1'),
 ('3:3.1', 'test 3.1')]

In [270]:
def _get_struct_fields(leaves):
    """
    Private function to generate the set of all SMA-X fields required to describe the
    tree structure of a nested dictionary.
    Args:
        leaves (list): List of leaf nodes from _recurse_nested_dict()

    Returns:
        list of (key, field, value, type) for each field at each level of the SMA-X nested structure.
                    type is either "value" for a leaf node, or "struct" for an intermediate node.
    """
    outpairs = []
    for l in leaves:
        if ":" in l[0]:
            tiers = l[0].split(":")
            for t, tier in enumerate(tiers[:-1]):
                superstruct = ":".join(tiers[0:t])
                pair = (superstruct, tiers[t], ":".join(tiers[0:t+1]), "struct")
                if pair not in outpairs:
                    outpairs.append(pair)
        tablekey = l[0].rsplit(":", 1)
        if len(tablekey) == 1:
            table = ""
            key = tablekey[0]
        else:
            table = tablekey[0]
            key = tablekey[1]
        outpairs.append((table, key, l[1], "value"))
    return outpairs


In [271]:
outpairs = _get_struct_fields(leaves)
outpairs

[('', '1', 'test 1', 'value'),
 ('', '2', '2', 'struct'),
 ('2', '2.1', 'test 2.1', 'value'),
 ('2', '2.2', '2:2.2', 'struct'),
 ('2:2.2', '2.2.1', 'test 2.2.1', 'value'),
 ('2:2.2', '2.2.2', 'test 2.2.2', 'value'),
 ('2', '2.3', '2:2.3', 'struct'),
 ('2:2.3', '2.3.1', 'test 2.3.1', 'value'),
 ('', '3', '3', 'struct'),
 ('3', '3.1', 'test 3.1', 'value')]

In [272]:
def _get_struct_tables(table, key, fields):
    """
    Private function to generate a dictionary of tables and arguments to HMSET_WITH_META calls from
    fields generated by _get_struct_fields() for a nested SMA-X struct.
    Args:
        table (str)   : The top level hash table name for the nested struct.
        key (str)     : The top level key for the nested struct.
        fields (list) : List of nested fields from _get_struct_fields()

    Returns:
        list of (key, field, value, type) for each field at each level of the SMA-X nested structure.
                    type is either "value" for a leaf node, or "struct" for an intermediate node.
    """
    tables = {}
    for field in fields:
        if field[3] == "struct":
            converted_data, type_name, dim = (":".join((table, key, field[2])), field[3], 1)
        else:
            converted_data, type_name, dim = src._to_smax_format(field[2])
        if field[0] != "":
            tab = f"{key}:{field[0]}"
        else:
            tab = key
        if tab not in tables:
            tables[tab] = []
        tables[tab].extend([field[1], converted_data, type_name, dim])
    return tables

In [275]:
tables = get_tables("test_share_struct", "test_key", outpairs)

In [276]:
for item in tables:
    print(item, tables[item])

test_key ['1', 'test 1', 'str', 1, '2', 'test_share_struct:test_key:2', 'struct', 1, '3', 'test_share_struct:test_key:3', 'struct', 1]
test_key:2 ['2.1', 'test 2.1', 'str', 1, '2.2', 'test_share_struct:test_key:2:2.2', 'struct', 1, '2.3', 'test_share_struct:test_key:2:2.3', 'struct', 1]
test_key:2:2.2 ['2.2.1', 'test 2.2.1', 'str', 1, '2.2.2', 'test 2.2.2', 'str', 1]
test_key:2:2.3 ['2.3.1', 'test 2.3.1', 'str', 1]
test_key:3 ['3.1', 'test 3.1', 'str', 1]


In [277]:
for k in tables.keys():
    print(k, *tables[k])

test_key 1 test 1 str 1 2 test_share_struct:test_key:2 struct 1 3 test_share_struct:test_key:3 struct 1
test_key:2 2.1 test 2.1 str 1 2.2 test_share_struct:test_key:2:2.2 struct 1 2.3 test_share_struct:test_key:2:2.3 struct 1
test_key:2:2.2 2.2.1 test 2.2.1 str 1 2.2.2 test 2.2.2 str 1
test_key:2:2.3 2.3.1 test 2.3.1 str 1
test_key:3 3.1 test 3.1 str 1


In [ ]:
{'test_key': [
    '1', 'test 1', 'str', 1, 
    '2', 'test_share_struct_dic:test_key:2', 'struct', 1, 
    '3', 'test_share_struct_dic:test_key:3', 'struct', 1], 
 'test_key:2': [
     '2.1', 'test 2.1', 'str', 1, 
     '2.2', 'test_share_struct_dic:test_key:2:2.2', 'struct', 1, 
     '2.3', 'test_share_struct_dic:test_key:2:2.3', 'struct', 1], 
 'test_key:2:2.2': [
     '2.2.1', 'test 2.2.1', 'str', 1, 
     '2.2.2', 'test 2.2.2', 'str', 1], 
 'test_key:2:2.3': [
     '2.3.1', 'test 2.3.1', 'str', 1], 
 'test_key:3': ['3.1', 'test 3.1', 'str', 1]
}

In [ ]:
{'test_key': [
    '1', 'test 1', 'str', 1, 
    '2', 'test_share_struct_dic:test_key:2', 'struct', 1, 
    '3', 'test_share_struct_dic:test_key:3', 'struct', 1], 
 'test_key:2': [
     '2.1', 'test 2.1', 'str', 1, 
     '2.2', 'test_share_struct_dic:test_key:2:2.2', 'struct', 1, 
     '2.3', 'test_share_struct_dic:test_key:2:2.3', 'struct', 1], 
 'test_key:2:2.2': [
     '2.2.1', 'test 2.2.1', 'str', 1, 
     '2.2.2', 'test 2.2.2', 'str', 1], 
 'test_key:2:2.3': [
     '2.3.1', 'test 2.3.1', 'str', 1], 
 'test_key:3': ['3.1', 'test 3.1', 'str', 1, 'T']
}

# Subclassing multiple types

In [309]:
class SmaxMixin(object):
    def __init__(self, smax_type=None, smax_dim=None, smax_date=None, smax_origin=None, smax_seq=None, smax_name=None):
        self.smax_type = smax_type
        self.smax_dim = smax_dim
        self.smax_date = smax_date
        self.smax_origin = smax_origin
        self.smax_seq = smax_seq
        self.smax_name = smax_name
    

In [474]:
class SmaxInt(int):
    def __new__(cls, *args, **kwargs):
        if 'base' in kwargs:
            return super().__new__(cls, args[0], kwargs['base'] )
        else:
            return super().__new__(cls, args[0])
        
    def __init__(self, *args, **kwargs):
        self.smax_type = None
        self.smax_dim = None
        self.smax_date = None
        self.smax_origin = None
        self.smax_seq = None
        self.smax_name = None
        
        smax_kwargs = dict((k,v) for k,v in kwargs.items() if k.startswith('smax_'))
        for kw in smax_kwargs:
            self.__setattr__(kw, smax_kwargs[kw])
        

In [476]:
a = SmaxInt(7)
print(a)
b = SmaxInt("17", base=8)
print(b)
c = SmaxInt(7, smax_type='int', smax_dim=1)
print(c)
print(c.smax_name)

7
15
7
None


In [477]:
class SmaxFloat(float):
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, args[0])
        
    def __init__(self, *args, **kwargs):
        self.smax_type = None
        self.smax_dim = None
        self.smax_date = None
        self.smax_origin = None
        self.smax_seq = None
        self.smax_name = None
        
        smax_kwargs = dict((k,v) for k,v in kwargs.items() if k.startswith('smax_'))
        for kw in smax_kwargs:
            self.__setattr__(kw, smax_kwargs[kw])

In [478]:
a = SmaxFloat(4.3)
print(a)
b = SmaxFloat(4.2, smax_type="float", smax_date=1661524418.794461)
print(b)
print(b.smax_name)

4.3
4.2
None


In [485]:
class SmaxStr(str):
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, *args)
        
    def __init__(self, *args, **kwargs):
        self.smax_type = None
        self.smax_dim = None
        self.smax_date = None
        self.smax_origin = None
        self.smax_seq = None
        self.smax_name = None
        
        smax_kwargs = dict((k,v) for k,v in kwargs.items() if k.startswith('smax_'))
        for kw in smax_kwargs:
            self.__setattr__(kw, smax_kwargs[kw])

In [489]:
a = SmaxStr("hello")
print(a)
print(a.smax_name)

a = SmaxStr("hello again", smax_name="my_test_table:my_test_key:a_string")
print(a)
print(a.smax_name)

hello
None
hello again
my_test_table:my_test_key:a_string


In [490]:
b = a

In [491]:
b.smax_name

'my_test_table:my_test_key:a_string'

In [492]:
b = 8

In [ ]:
b.